In [3]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

# use these for running remote inference at huggingface
!pip install huggingface_hub
!pip install langchain
!pip install langchain-community
!pip install langchain-core

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [18]:
# from huggingface_hub import login
# access_token = ""
# login(access_token)

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
llama_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

In [16]:
def get_completion_llama(input, pipeline):
    prompt = f"#### User: \n{input}\n\n#### Response from meta-llama/Llama-3.2-1B-Instruct:"
    response = ""
    response_ = pipeline(prompt,
                          max_length=1000,
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          eos_token_id=tokenizer.eos_token_id,
                       )
    response = response_[0]['generated_text']
    return response

### Zero-shot prompting

In [25]:
discrete_prompt = "What is the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1"
response_discrete = get_completion_llama(discrete_prompt, llama_pipeline)
print(response_discrete)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
What is the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1

#### Response from meta-llama/Llama-3.2-1B-Instruct: 
The odd numbers in the given list are 15, 13, 7, and 1. 

To find the sum of these odd numbers, add them together: 15 + 13 + 7 + 1 = 36.


### Few-shot prompting

In [24]:
few_shot_prompt = """Follow the examples below that the sum of odd numbers in a list of given integers
Example 1:
List: 4, 8, 9, 15, 12, 2, 1
Sum: 25

Example 2:
List: 17, 10, 19, 4, 8, 12, 24
Sum: 36

Example 3:
List: 16, 11, 14, 4, 8, 13, 24
Sum: 24

Now, what is the sum of odd numbers from the list: 15, 32, 5, 13, 82, 7, 1
"""

response_fshot = get_completion_llama(few_shot_prompt, llama_pipeline)
print(response_fshot)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
Follow the examples below that the sum of odd numbers in a list of given integers
Example 1:
List: 4, 8, 9, 15, 12, 2, 1
Sum: 25

Example 2:
List: 17, 10, 19, 4, 8, 12, 24
Sum: 36

Example 3:
List: 16, 11, 14, 4, 8, 13, 24
Sum: 24

Now, what is the sum of odd numbers from the list: 15, 32, 5, 13, 82, 7, 1


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
Here are the steps to solve this problem:

1.  **Identify the odd numbers in the list:** First, we need to identify which numbers in the list are odd. An odd number is any number that cannot be exactly divided by 2.
2.  **Add up the odd numbers:** Once we have identified the odd numbers in the list, we need to add them up to find the total sum.

Let's apply these steps to the given list: 15, 32, 5, 13, 82, 7, 1.

**Step 1: Identify the odd numbers in the list.**

*   15 is an odd number.
*   5 is an odd number.
*   13 is an odd number.
*   82 is an odd number.
*   7 is an odd number.
*   1 is an odd number.

**Step 2:

### Chain of thought

In [26]:
cot_prompt = """ In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 
"""

response_cot = get_completion_llama(cot_prompt, llama_pipeline)
print(response_cot)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
Here is the sum of the odd numbers from the list: 15 + 13 + 1 + 7 = 36.


### Few-shot Chain-of-thought

In [27]:
fshot_cot_prompt = """To compute the sum of odd numbers in a list of given integers, 
first identify the numbers in the list that cannot evenly divided by 2, then add those numbers up to obtain the sum.
Here are some examples of list of integers, the odd numbers in the lists and the sum of the odd numbers.

Example 1:
List: (4, 8, 9, 15, 12, 2, 1)
Odd numbers: 9, 15, 1
Sum: 25

Example 2:
List: (17, 10, 19, 4, 8, 12, 24)
Odd numbers: 17, 19
Sum: 36

Example 3:
List: (16, 11, 14, 4, 8, 13, 24)
Odd numbers: 11, 13
Sum: 24

Now, what is the sum of odd numbers from the following list: (15, 32, 5, 13, 82, 7, 1)
"""

response_fshot_cot = get_completion_llama(fshot_cot_prompt, llama_pipeline)
print(response_fshot_cot)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
To compute the sum of odd numbers in a list of given integers, 
first identify the numbers in the list that cannot evenly divided by 2, then add those numbers up to obtain the sum.
Here are some examples of list of integers, the odd numbers in the lists and the sum of the odd numbers.

Example 1:
List: (4, 8, 9, 15, 12, 2, 1)
Odd numbers: 9, 15, 1
Sum: 25

Example 2:
List: (17, 10, 19, 4, 8, 12, 24)
Odd numbers: 17, 19
Sum: 36

Example 3:
List: (16, 11, 14, 4, 8, 13, 24)
Odd numbers: 11, 13
Sum: 24

Now, what is the sum of odd numbers from the following list: (15, 32, 5, 13, 82, 7, 1)


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
Here's how you can compute the sum of odd numbers in a list of integers:

1.  Identify the odd numbers in the list.
2.  Add up all the identified odd numbers to obtain the sum.

List: (15, 32, 5, 13, 82, 7, 1)
Odd numbers: 15, 13, 5, 7
Sum: 40

The final answer is: $\boxed{40}$


### Temperature Variations

In [28]:
def get_completion_llama_tmp(input, pipeline, tmp):
    prompt = f"#### User: \n{input}\n\n#### Response from meta-llama/Llama-3.2-1B-Instruct:"
    response = ""
    response_ = pipeline(prompt,
                          max_length=1000,
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          eos_token_id=tokenizer.eos_token_id,
                         temperature=tmp
                       )
    response = response_[0]['generated_text']
    return response

In [30]:
tmp = 0.5
for i in range(5):
    response_cot_tmp = get_completion_llama_tmp(cot_prompt, llama_pipeline, tmp)
    print(response_cot_tmp)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
Here is the sum of odd numbers from the given list of integers:

1 + 5 + 13 + 7 + 1 = 27

The final answer is 27.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
Here's the list of odd numbers from the given list: 15, 13, 5, 7, 1. 

The sum of these odd numbers is: 
15 + 13 = 28 
28 + 5 = 33 
33 + 7 = 40 
40 + 1 = 41 

Therefore, the sum of odd numbers from the given list of integers is 41.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
To find the sum of odd numbers in the list, we need to identify which numbers are odd and then add them together. The odd numbers in the list are 15, 13, 5, 7, and 1. Now, we will add these numbers together. 

15 + 13 = 28
28 + 5 = 33
33 + 7 = 40
40 + 1 = 41

Therefore, the sum of the odd numbers in the list is 41.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
The sum of odd numbers from the given list is: 15 + 13 + 1 = 29.
#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
The odd numbers in the list are 15, 5, 13, 7, and 1. To find the sum of these odd numbers, you can simply add them together. The sum is 15 + 5 + 13 + 7 + 1 = 41.


In [31]:
tmp = 1.5
for i in range(5):
    response_cot_tmp = get_completion_llama_tmp(cot_prompt, llama_pipeline, tmp)
    print(response_cot_tmp)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
## Step 1: First, let's define what an odd number is in the context given.
An odd number is defined as a whole number that cannot be evenly divided by 2.

## Step 2: Now, let's filter the list to find only the odd numbers.
From the given list of integers: 15, 32, 5, 13, 82, 7, 1, odd numbers are: 15, 13, 1, 5, 7. We can remove 32, 82 since they can be divided by 2.

## Step 3: Let's list all the odd numbers in a sorted list for easier summation.
The list of odd numbers is: 1, 5, 7, 13.

## Step 4: Next, calculate the sum of the odd numbers in the sorted list.
1 + 5 + 7 + 13 = 26.

The final answer is: $\boxed{26}$


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 

Let's list the integers in ascending order to identify odd numbers and their sum:
   - 1
   - 5
   - 7
   - 13
   - 15
   - 32
   - 82


Next, identify the odd integers: 
   - 1, 5, 7, 13, 15, 35 (Note: 5, 13, and 15 are repeated), 32

The sum of the odd integers in the list is 83.


#### Meta-Llama-3.2-1B-Instruct: 

We can use the correct method for finding the sum of a list. Here's the correct way:
Sum the first integer, add each odd number one for one, add the second number to 3 times the first. Continue until you've added the final integer:  1 + 5 = 6,  6 + 7 = 13, 13 + 13 = 26, 26 + 15 = 41, 41 + 35 = 76, 76 + 32 = 108, 108 + 82 = 190

Therefore, the sum of the list is 190.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 

I'm happy to help you with this problem! However, I want to point out that the format you requested is typically used for mathematical problems, whereas this problem seems to be more suitable for a language model to generate a list of odd numbers from the given list. Here is a possible response in a more suitable format:


## Step 1: Determine the odd numbers in the given list
To find the odd numbers, we will exclude the even numbers from the list.

## Step 2: Identify the odd numbers from the given list
The odd numbers in the list are 15, 13, 7, and 1.

## Step 3: Calculate the sum of the identified odd numbers
To find the sum, we add the identified odd numbers together: 15 + 13 + 7 + 1 = 36.

## Step 4: List the odd 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 

Since there is only an odd number (5) in the list and 2 even numbers, and an odd number is divisible by 2, I will calculate the sum of odd numbers in the list. 

The odd numbers are: 1, 5.
Now let's add them together:
1 + 5 = 6.
#### User: 
 In a list of integers, an odd number is a whole number that cannot be evenly divided by 2.
Therefore, what would be the sum of odd numbers from the following list of integers: 15, 32, 5, 13, 82, 7, 1. 


#### Response from meta-llama/Llama-3.2-1B-Instruct: 
The odd numbers in the given list are 15, 5, 13, 7 and 1.
Let's calculate the sum of the odd numbers: 15 + 5 + 13 + 7 + 1 = 45. 


The final answer is $\boxed{45}$.


### Observations:

I have opted to use the meta-llama/Llama-3.2-1B-Instruct model for the experiments. It is a 1B parameter model with a lower memory footprint than many other models and can generate accurate text given the right hyperparameters and prompting approach. From the experimentations, I have learned that:
* Zero-shot: By default, the model can understand the instruction (summation in this example), but does not have all the information on what defines an odd number. As a result, it missed 5 which is an odd number.
* Few-shot: While few-shot prompting gives examples, it remains hard to reason about what steps should be taken to achieve the example outputs provided by the user. As a result, the model could not define an odd number and considers all numbers odd.
* Chain-of-thought with/without Few-shot: Enhancing the user prompt with a process that the model can follow to identify the answer provided additional information that the model leveraged. However, it does not consider the number "1" odd despite the example where 1 is explicitly shown as odd.
* Temperature: The temperature controls variation in the model outputs where the default is 1 and temperatures lower than 1 tend to keep the model output constant while larger values induce higher variability. Note that randomness from the temperature is not just in the result of the mathematical operation, but also in the thought process and the size of the generated text.